## Mounting Drive ke Google Colab agar dapat mengakses data yang sudah disimpan di dalam drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import module dan install library tambahan

In [ ]:
# import module yang diperlukan untuk pre-processing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import spectrogram
import os
import shutil
import gc
from PIL import Image
import pickle


# install module tambahan
!pip install mne PyWavelets

import mne
import pywt

## Simpan data pada dataset dalam variabel

In [ ]:
# Dataset file seizure/kejang yang telah dimodifikasi menurut nama file dan
# data nya (terdapat pengulangan nama file yang menandakan munculnya seizure/kejang pada
# 1 file dengan indeks/lokasi waktu yang berbeda)
# list untuk penamaan file saat proses penyimpanan pada tahap berikutnya
saved_seizure_file_name = pd.read_csv('/content/drive/MyDrive/skripsi/chbDataset/RECORDS-WITH-SEIZURES_filenames.csv').to_numpy().flatten()
# list nama .edf file yang diproses sesuai dengan penamaan file nya
seizure_files = pd.read_csv('/content/drive/MyDrive/skripsi/chbDataset/RECORDS-WITH-SEIZURES_files.csv').to_numpy().flatten()

# Dataset dengan indikasi kejang
file_with_seizure = pd.read_csv('/content/drive/MyDrive/skripsi/chbDataset/RECORDS-WITH-SEIZURES.csv').to_numpy().flatten()
# Dataset lengkap tanpa modifikasi
all_files = pd.read_csv('/content/drive/MyDrive/skripsi/chbDataset/RECORDS.csv').to_numpy().flatten()
# Dataset tanpa indikasi kejang
file_not_seizure = [element for element in all_files if element not in file_with_seizure]

## Transformasi dataset berdasarkan metode dan urutan waktu

In [ ]:
# Import data waktu yang digunakan untuk memetakan lokasi seizure/kejang pada data sinyal

with open("/content/drive/MyDrive/skripsi/10SecondTime.pickle", "rb") as f:
  time_10 = pickle.load(f)
  f.close()

with open("/content/drive/MyDrive/skripsi/20SecondTime.pickle", "rb") as f:
  time_20 = pickle.load(f)
  f.close()

with open("/content/drive/MyDrive/skripsi/30SecondTime.pickle", "rb") as f:
  time_30 = pickle.load(f)
  f.close()

In [ ]:
print(time_10)

In [ ]:
# Transformasi data sinyal menjadi citra
# Seizure/Kejang

figSizeX = 2.56
figSizeY = 2.56


os.makedirs("/content/drive/MyDrive/skripsi/10Second", exist_ok=True)
# proses konversi ke bentuk gambar
indexTime = 0
for file in saved_seizure_file_name[0:1]:
  # menentukan dan membuat nama folder yang menjadi tempat penyimpanan hasil konversi
  hasil = os.path.splitext(file)
  fileName = hasil[0].split('/')
  os.makedirs('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/seizureFolder/' + hasil[0], exist_ok=True)
  # pemrosesan data pada .edf file berdasarkan nilai indexTime
  print(seizure_files[indexTime])
  raw = mne.io.read_raw_edf('/content/drive/MyDrive/skripsi/chbDataset/' + seizure_files[indexTime], preload=True)
  selected_channels = raw.ch_names
  isFileExist = os.path.isdir('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/seizureFolder/' + hasil[0])
  #cek apakah folder sudah ada atau belum
  if isFileExist == False:
    os.makedirs('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/seizureFolder/' + hasil[0], exist_ok=True)
  for ch in selected_channels:
    print(indexTime, ch)
    data, times = raw[ch]
    np.set_printoptions(suppress=True)
    # inisialisasi list untuk menyimpan data sinyal dan data waktu secara berurutan
    collectionOfData = []
    collectionOfTime = []
    result = dict(enumerate(times.flatten(), 1))
    for index in result:
      if result[index] >= time_10[indexTime][0] and result[index] <= time_10[indexTime][1]:
        collectionOfData.append(data.T[index][0])
        collectionOfTime.append(times[index])
    collectionOfData = np.array(collectionOfData)
    collectionOfTime = np.array(collectionOfTime)

    heatmap, xedges, yedges = np.histogram2d(collectionOfTime, collectionOfData, bins=50)

    hist, bin_edges = np.histogram(collectionOfData, bins=50)

    scales = np.arange(1,128)

    X, Y = np.meshgrid(collectionOfTime, scales)

    cwt_coefficients, frequencies = pywt.cwt(collectionOfData, scales, 'mexh')

    fft_frequencies, fft_times, Sxx = spectrogram(collectionOfData, 257)

    # Plot Wavelet Transform
    fig, axs = plt.subplots(1, 1, figsize=(figSizeX, figSizeY))
    axs.pcolormesh(X, Y, cwt_coefficients, shading='gouraud', cmap='viridis')
    axs.set_xticks([])
    axs.set_yticks([])
    fig.savefig('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/seizureFolder/'+ hasil[0] +'/'+ fileName[1] +'_wavelet_'+ ch +'.png')
    plt.cla()
    plt.clf()
    # Plot FFT Spectrogram
    fig1, axs1 = plt.subplots(1, 1, figsize=(figSizeX, figSizeY))
    axs1.pcolormesh(fft_times, fft_frequencies, 10 * np.log10(Sxx), shading='gouraud', cmap='plasma')
    axs1.set_xticks([])
    axs1.set_yticks([])
    fig1.savefig('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/seizureFolder/'+ hasil[0] +'/'+ fileName[1] +'_spectogram_'+ ch +'.png')
    plt.cla()
    plt.clf()
    # Plot Heatmap
    fig2, axs2 = plt.subplots(1, 1, figsize=(figSizeX, figSizeY))
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    axs2.imshow(heatmap.T, extent=extent, origin='lower', cmap='hot', aspect='auto')
    axs2.set_xticks([])
    axs2.set_yticks([])
    fig2.savefig('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/seizureFolder/'+ hasil[0] +'/'+ fileName[1] +'_heatmap_'+ ch +'.png')
    plt.cla()
    plt.clf()

    del fig,axs,fig1,axs1,fig2,axs2,fft_frequencies, fft_times, Sxx,cwt_coefficients, frequencies,X, Y,scales,hist, bin_edges,
    heatmap, xedges, yedges,collectionOfData,collectionOfTime,data, times,result
  indexTime += 1
  raw.close()
  del hasil,fileName,raw,selected_channels
plt.close('all')
gc.collect()

In [ ]:
#Transformasi data sinyal menjadi citra
# Normal

figSizeX = 2.56
figSizeY = 2.56

indexTime = 0
for file in file_not_seizure[0:1]:
  hasil = os.path.splitext(file)
  fileName = hasil[0].split('/')
  os.makedirs('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/notSeizureFolder/' + hasil[0], exist_ok=True)
  raw = mne.io.read_raw_edf('/content/drive/MyDrive/skripsi/chbDataset/' + file, preload=True)
  selected_channels = raw.ch_names
  isFileExist = os.path.isdir('content/drive/MyDrive/skripsi/10Second/hasilKonversi/notSeizureFolder/' + hasil[0])
  if isFileExist == False:
    os.makedirs('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/notSeizureFolder/' + hasil[0], exist_ok=True)
  for ch in selected_channels:
    print(indexTime, ch)
    data, times = raw[ch]
    np.set_printoptions(suppress=True)
    collectionOfData = []
    collectionOfTime = []
    result = dict(enumerate(times.flatten(), 1))
    for index in result:
      if result[index] >= 0 and result[index] <= 12:
        collectionOfData.append(data.T[index][0])
        collectionOfTime.append(times[index])
    collectionOfData = np.array(collectionOfData)
    collectionOfTime = np.array(collectionOfTime)

    heatmap, xedges, yedges = np.histogram2d(collectionOfTime, collectionOfData, bins=50)

    hist, bin_edges = np.histogram(collectionOfData, bins=50)

    scales = np.arange(1,128)

    X, Y = np.meshgrid(collectionOfTime, scales)

    cwt_coefficients, frequencies = pywt.cwt(collectionOfData, scales, 'mexh')

    fft_frequencies, fft_times, Sxx = spectrogram(collectionOfData, 257)

    # Plot Wavelet Transform
    fig, axs = plt.subplots(1, 1, figsize=(figSizeX, figSizeY))
    axs.pcolormesh(X, Y, cwt_coefficients, shading='gouraud', cmap='viridis')
    axs.set_xticks([])
    axs.set_yticks([])
    fig.savefig('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/notSeizureFolder/'+ hasil[0] +'/'+ fileName[1] +'_wavelet_'+ ch +'.png')
    plt.cla()
    plt.clf()
    # Plot FFT Spectrogram
    fig1, axs1 = plt.subplots(1, 1, figsize=(figSizeX, figSizeY))
    axs1.pcolormesh(fft_times, fft_frequencies, 10 * np.log10(Sxx), shading='gouraud', cmap='plasma')
    axs1.set_xticks([])
    axs1.set_yticks([])
    fig1.savefig('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/notSeizureFolder/'+ hasil[0] +'/'+ fileName[1] +'_spectogram_'+ ch +'.png')
    plt.cla()
    plt.clf()
    # Plot Heatmap
    fig2, axs2 = plt.subplots(1, 1, figsize=(figSizeX, figSizeY))
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    axs2.imshow(heatmap.T, extent=extent, origin='lower', cmap='hot', aspect='auto')
    axs2.set_xticks([])
    axs2.set_yticks([])
    fig2.savefig('/content/drive/MyDrive/skripsi/10Second/hasilKonversi/notSeizureFolder/'+ hasil[0] +'/'+ fileName[1] +'_heatmap_'+ ch +'.png')
    plt.cla()
    plt.clf()
    del fig,axs,fig1,axs1,fig2,axs2,fft_frequencies, fft_times, Sxx,cwt_coefficients, frequencies,X, Y,scales,hist, bin_edges,
    heatmap, xedges, yedges,collectionOfData,collectionOfTime,data, times,result
  indexTime += 1
  raw.close()
  del hasil,fileName,raw,selected_channels
plt.close('all')
gc.collect()

## Pengelompokkan data berdasarkan metode nya

In [ ]:
# Seizure/Kejang

result = os.listdir("/content/drive/MyDrive/skripsi/10Second/hasilKonversi/seizureFolder/")
for i in result:
  if "chb" in i:
    directory = "/content/drive/MyDrive/skripsi/10Second/hasilKonversi/seizureFolder/" + i
    count = 0
    # Iterate directory
    for path in os.listdir(directory):
        # check if current path is a file
        if os.path.isdir(os.path.join(directory, path)):
            count += 1
    for index in range(count):
      hasil = [x[0] for x in os.walk(directory)][index + 1]
      for file in os.listdir(hasil):
        fileName = os.path.join(hasil, file)
        if("heatmap" in fileName):
          os.makedirs("/content/drive/MyDrive/skripsi/10Second/hasilKelompok/seizureFolder/heatmap", exist_ok=True)
          shutil.copy(fileName, "/content/drive/MyDrive/skripsi/10Second/hasilKelompok/seizureFolder/heatmap/")
        elif("spectogram" in fileName):
          os.makedirs("/content/drive/MyDrive/skripsi/10Second/hasilKelompok/seizureFolder/spectogram", exist_ok=True)
          shutil.copy(fileName, "/content/drive/MyDrive/skripsi/10Second/hasilKelompok/seizureFolder/spectogram/")
        elif("wavelet" in fileName):
          os.makedirs("/content/drive/MyDrive/skripsi/10Second/hasilKelompok/seizureFolder/wavelet", exist_ok=True)
          shutil.copy(fileName, "/content/drive/MyDrive/skripsi/10Second/hasilKelompok/seizureFolder/wavelet/")

In [ ]:
# Normal
result = os.listdir("/content/drive/MyDrive/skripsi/10Second/hasilKonversi/notSeizureFolder/")
for i in result:
  if "chb" in i:
    directory = "/content/drive/MyDrive/skripsi/10Second/hasilKonversi/notSeizureFolder/" + i
    count = 0
    # Iterate directory
    for path in os.listdir(directory):
        # check if current path is a file
        if os.path.isdir(os.path.join(directory, path)):
            count += 1
    for index in range(count):
      hasil = [x[0] for x in os.walk(directory)][index + 1]
      for file in os.listdir(hasil):
        fileName = os.path.join(hasil, file)
        if("heatmap" in fileName):
          os.makedirs("/content/drive/MyDrive/skripsi/10Second/hasilKelompok/notSeizureFolder/heatmap", exist_ok=True)
          shutil.copy(fileName, "/content/drive/MyDrive/skripsi/10Second/hasilKelompok/notSeizureFolder/heatmap/")
        elif("spectogram" in fileName):
          os.makedirs("/content/drive/MyDrive/skripsi/10Second/hasilKelompok/notSeizureFolder/spectogram", exist_ok=True)
          shutil.copy(fileName, "/content/drive/MyDrive/skripsi/10Second/hasilKelompok/notSeizureFolder/spectogram/")
        elif("wavelet" in fileName):
          os.makedirs("/content/drive/MyDrive/skripsi/10Second/hasilKelompok/notSeizureFolder/wavelet", exist_ok=True)
          shutil.copy(fileName, "/content/drive/MyDrive/skripsi/10Second/hasilKelompok/notSeizureFolder/wavelet/")

## Konversi channel gambar dari 4 channel ke 3 channel

In [ ]:
# fungsi untuk merubah channel pada citra
def cmyk_to_rgb(input_path, output_path):
    # Open the CMYK image using Pillow
    cmyk_image = Image.open(input_path)
    # Resize the image to 256 x 256 dimension
    resize = cmyk_image.resize((256, 256))
    # Convert CMYK to RGB
    rgb_image = resize.convert("RGB")
    # Save the RGB image
    rgb_image.save(output_path)

In [ ]:
# Seizure/Kejang
result = os.listdir("/content/drive/MyDrive/skripsi/10Second/hasilKelompok/seizureFolder/")
for item in result:
  directory = "/content/drive/MyDrive/skripsi/10Second/hasilKelompok/seizureFolder/" + item + "/"
  os.makedirs("/content/drive/MyDrive/skripsi/10Second/hasilUbahChannel/seizureFolder/" + item + "/", exist_ok=True)
  for path in os.listdir(directory):
    file = os.path.join(directory, path)
    input_file_path = file  # Replace with your CMYK image file path
    output_file_path = "/content/drive/MyDrive/skripsi/10Second/hasilUbahChannel/seizureFolder/" + item + "/" + path  # Replace with the desired output file path
    cmyk_to_rgb(input_file_path, output_file_path)

In [ ]:
# Normal
result = os.listdir("/content/drive/MyDrive/skripsi/10Second/hasilKelompok/notSeizureFolder/")
for item in result:
  directory = "/content/drive/MyDrive/skripsi/10Second/hasilKelompok/notSeizureFolder/" + item + "/"
  os.makedirs("/content/drive/MyDrive/skripsi/10Second/hasilUbahChannel/notSeizureFolder/" + item + "/", exist_ok=True)
  for path in os.listdir(directory):
    file = os.path.join(directory, path)
    input_file_path = file  # Replace with your CMYK image file path
    output_file_path = "/content/drive/MyDrive/skripsi/10Second/hasilUbahChannel/notSeizureFolder/" + item + "/" + path  # Replace with the desired output file path
    cmyk_to_rgb(input_file_path, output_file_path)

## Memberikan label pada data seizure dan normal, lalu menggabungkan keduanya

In [ ]:
import random
from sklearn import preprocessing
import sys

nama = "wavelet"

In [ ]:
dataset = []
seizure_path = f'/content/drive/MyDrive/skripsi/10Second/hasilUbahChannel/seizureFolder/{nama}/'
not_seizure_path = f'/content/drive/MyDrive/skripsi/10Second/hasilUbahChannel/notSeizureFolder/{nama}/'


for seizure in os.listdir(seizure_path):
  if(seizure == 'desktop.ini'):
    continue
  img = np.asarray(Image.open(os.path.join(seizure_path,seizure)))
  dataset.append([img, "Seizure"])

for not_seizure in os.listdir(not_seizure_path):
  if(not_seizure == 'desktop.ini'):
    continue
  img = np.asarray(Image.open(os.path.join(not_seizure_path,not_seizure)))
  dataset.append([img, "Tidak Seizure"])


for i in range(0,10):
  random.shuffle(dataset)

train_images = []
train_labels = []
for i in range(len(dataset)):
  train_images.append(dataset[i][0])
  train_labels.append(dataset[i][1])

In [ ]:
print(train_images)
print(train_labels)

In [ ]:
encoder = preprocessing.LabelEncoder()
encoder.fit(train_labels)
# Print the mapping
print("\nLabel mapping:")
for i, item in enumerate(encoder.classes_): print(item, '-->', i)

In [ ]:
# Encode a set of labels using the encoder
encoded_values = encoder.transform(train_labels)
decoded_list = encoder.inverse_transform(encoded_values)
np.set_printoptions(threshold=sys.maxsize)
print(encoded_values)
print(decoded_list)
np.save(f'/content/drive/MyDrive/skripsi/{nama}_classes.npy', encoder.classes_)

In [ ]:
with open(f'/content/drive/MyDrive/skripsi/{nama}_batch', 'wb+') as f:
    pickle.dump({b"data": train_images, b"labels": encoded_values}, f)
    f.close()

In [ ]:
dataset = []
seizure_path_10 = f'/content/dataset/10second/seizureFolder/{nama}/'
not_seizure_path_10 = f'/content/dataset/10second/notSeizureFolder/{nama}/'


for seizure in os.listdir(seizure_path_10):
  if(seizure == 'desktop.ini'):
    continue
  img = np.asarray(Image.open(os.path.join(seizure_path_10,seizure)))
  dataset.append([img, "Seizure"])

for not_seizure in os.listdir(not_seizure_path_10):
  if(not_seizure == 'desktop.ini'):
    continue
  img = np.asarray(Image.open(os.path.join(not_seizure_path_10,not_seizure)))
  dataset.append([img, "Tidak Seizure"])


for i in range(0,10):
  random.shuffle(dataset)

train_images = []
train_labels = []
for i in range(len(dataset)):
  train_images.append(dataset[i][0])
  train_labels.append(dataset[i][1])

In [ ]:
# Pengubahan label menjadi numerik
# tahap ini digunakan untuk mengubah label menjadi numerik untuk mempercepat proses komputasi
encoder = preprocessing.LabelEncoder()
encoder.fit(train_labels)
# Print the mapping
print("\nLabel mapping:")
for i, item in enumerate(encoder.classes_): print(item, '-->', i)

In [ ]:
# Proses menyimpan class dari label yang sudah ditentukan
encoded_values = encoder.transform(train_labels)
decoded_list = encoder.inverse_transform(encoded_values)
np.set_printoptions(threshold=sys.maxsize)
np.save(f'/content/drive/MyDrive/skripsi/eeg/{nama}_10_second_classes.npy', encoder.classes_)

In [ ]:
with open(f'/content/drive/MyDrive/skripsi/eeg/{nama}_10_second_batch', 'wb+') as f:
    pickle.dump({b"data": train_images, b"labels": encoded_values}, f)
    f.close()

In [ ]:
dataset = []
seizure_path_20 = f'/content/dataset/20second/seizureFolder/{nama}/'
not_seizure_path_20 = f'/content/dataset/20second/notSeizureFolder/{nama}/'

for seizure in os.listdir(seizure_path_20):
  if(seizure == 'desktop.ini'):
    continue
  img = np.asarray(Image.open(os.path.join(seizure_path_20,seizure)))
  dataset.append([img, "Seizure"])

for not_seizure in os.listdir(not_seizure_path_20):
  if(not_seizure == 'desktop.ini'):
    continue
  img = np.asarray(Image.open(os.path.join(not_seizure_path_20,not_seizure)))
  dataset.append([img, "Tidak Seizure"])


for i in range(0,10):
  random.shuffle(dataset)

train_images = []
train_labels = []
for i in range(len(dataset)):
  train_images.append(dataset[i][0])
  train_labels.append(dataset[i][1])


In [ ]:
encoder = preprocessing.LabelEncoder()
encoder.fit(train_labels)
# Print the mapping
print("\nLabel mapping:")
for i, item in enumerate(encoder.classes_): print(item, '-->', i)

In [ ]:
# Encode a set of labels using the encoder
encoded_values = encoder.transform(train_labels)
decoded_list = encoder.inverse_transform(encoded_values)
np.set_printoptions(threshold=sys.maxsize)
np.save(f'/content/drive/MyDrive/skripsi/eeg/{nama}_20_second_classes.npy', encoder.classes_)

In [ ]:
with open(f'/content/drive/MyDrive/skripsi/eeg/{nama}_20_second_batch', 'wb+') as f:
    pickle.dump({b"data": train_images, b"labels": encoded_values}, f)
    f.close()

In [ ]:
dataset = []
seizure_path_30 = f'/content/dataset/30second/seizureFolder/{nama}/'
not_seizure_path_30 = f'/content/dataset/30second/notSeizureFolder/{nama}/'


for seizure in os.listdir(seizure_path_30):
  if(seizure == 'desktop.ini'):
    continue
  img = np.asarray(Image.open(os.path.join(seizure_path_30,seizure)))
  dataset.append([img, "Seizure"])

for not_seizure in os.listdir(not_seizure_path_30):
  if(not_seizure == 'desktop.ini'):
    continue
  img = np.asarray(Image.open(os.path.join(not_seizure_path_30,not_seizure)))
  dataset.append([img, "Tidak Seizure"])

for i in range(0,10):
  random.shuffle(dataset)

train_images = []
train_labels = []
for i in range(len(dataset)):
  train_images.append(dataset[i][0])
  train_labels.append(dataset[i][1])



In [ ]:
encoder = preprocessing.LabelEncoder()
encoder.fit(train_labels)
# Print the mapping
print("\nLabel mapping:")
for i, item in enumerate(encoder.classes_): print(item, '-->', i)

In [ ]:
# Encode a set of labels using the encoder
encoded_values = encoder.transform(train_labels)
decoded_list = encoder.inverse_transform(encoded_values)
np.set_printoptions(threshold=sys.maxsize)
np.save(f'/content/drive/MyDrive/skripsi/eeg/{nama}_30_second_classes.npy', encoder.classes_)

In [ ]:
with open(f'/content/drive/MyDrive/skripsi/eeg/{nama}_30_second_batch', 'wb+') as f:
    pickle.dump({b"data": train_images, b"labels": encoded_values}, f)
    f.close()

## Pembuatan Model

In [ ]:
import numpy as np
import os
import keras
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(256, 256, 3)))
model.add(keras.layers.Conv2D(128, (5, 5), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(256, (5, 5), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(512, (5, 5), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

# Pengujian model

In [ ]:
# meng-copy data batch yang sudah disimpan ke environment google colab untuk mempercepat pengaksesan file

!cp '/content/drive/MyDrive/skripsi/eeg/wavelet_10_second_batch' '/content'

In [ ]:
# Membaca file batch sesuai dengan durasi yang ditentukan
with open('/content/wavelet_10_second_batch', 'rb') as f:
  wavelet_10 = pickle.load(f)

In [ ]:
# Mengelompokkan data dan label
wavelet_10_data = np.array(wavelet_10[b'data'])
wavelet_10_labels = np.array(wavelet_10[b'labels'])

In [ ]:
# pembagian data latih dan data uji
wavelet_10_train_data, wavelet_10_test_data, wavelet_10_train_labels, wavelet_10_test_labels = train_test_split(wavelet_10_data, wavelet_10_labels, test_size=0.2)


In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(wavelet_10_train_data,wavelet_10_train_labels, epochs=50, validation_split=0.2)

In [ ]:
# Menampilkan hasil dari proses pelatihan model CNN
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

wavelet_10_test_loss, wavelet_10_test_acc = model.evaluate(wavelet_10_test_data,  wavelet_10_test_labels)

In [ ]:
# Menampilkan hasil tes akurasi
print(wavelet_10_test_acc)

In [ ]:
# Menyimpan histori dari proses pelatihan model CNN
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/wavelet_10_history.csv')

In [ ]:
# Menyimpan model ke dalam file agar bisa digunakan kembali
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/wavelet_10_model.keras')

In [ ]:
del wavelet_10_train_data, wavelet_10_test_data, wavelet_10_train_labels, wavelet_10_test_labels
del history

In [ ]:
# Meng-copy file ke dalam environment google colab agar mempercepat pengaksesan file
!cp '/content/drive/MyDrive/skripsi/eeg/wavelet_20_second_batch' '/content'

In [ ]:
with open('/content/wavelet_20_second_batch', 'rb') as f:
  wavelet_20 = pickle.load(f)

In [ ]:
wavelet_20_data = np.array(wavelet_20[b'data'])
wavelet_20_labels = np.array(wavelet_20[b'labels'])

In [ ]:
wavelet_20_train_data, wavelet_20_test_data, wavelet_20_train_labels, wavelet_20_test_labels = train_test_split(wavelet_20_data, wavelet_20_labels, test_size=0.2)

In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(wavelet_20_train_data,wavelet_20_train_labels, epochs=50, validation_split=0.2)

In [ ]:
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

wavelet_20_test_loss, wavelet_20_test_acc = model.evaluate(wavelet_20_test_data,  wavelet_20_test_labels)

In [ ]:
print(wavelet_20_test_acc)

In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/wavelet_20_history.csv')

In [ ]:
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/wavelet_20_model.keras')

In [ ]:
del wavelet_20_train_data, wavelet_20_test_data, wavelet_20_train_labels, wavelet_20_test_labels
del history

In [ ]:
!cp '/content/drive/MyDrive/skripsi/eeg/wavelet_30_second_batch' '/content'

In [ ]:
with open('/content/wavelet_30_second_batch', 'rb') as f:
  wavelet_30 = pickle.load(f)

In [ ]:
wavelet_30_data = np.array(wavelet_30[b'data'])
wavelet_30_labels = np.array(wavelet_30[b'labels'])

In [ ]:
wavelet_30_train_data, wavelet_30_test_data, wavelet_30_train_labels, wavelet_30_test_labels = train_test_split(wavelet_30_data, wavelet_30_labels, test_size=0.2)

In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(wavelet_30_train_data,wavelet_30_train_labels, epochs=50, validation_split=0.2)

In [ ]:
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

wavelet_30_test_loss, wavelet_30_test_acc = model.evaluate(wavelet_30_test_data,  wavelet_30_test_labels)

In [ ]:
print(wavelet_30_test_acc)

In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/wavelet_30_history.csv')

In [ ]:
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/wavelet_30_model.keras')

In [ ]:
del wavelet_30_train_data, wavelet_30_test_data, wavelet_30_train_labels, wavelet_30_test_labels
del history

In [ ]:
!cp '/content/drive/MyDrive/skripsi/eeg/spectogram_10_second_batch' '/content'

In [ ]:
with open('/content/spectogram_10_second_batch', 'rb') as f:
  spectogram_10 = pickle.load(f)

In [ ]:
spectogram_10_data = np.array(spectogram_10[b'data'])
spectogram_10_labels = np.array(spectogram_10[b'labels'])

In [ ]:
spectogram_10_train_data, spectogram_10_test_data, spectogram_10_train_labels, spectogram_10_test_labels = train_test_split(spectogram_10_data, spectogram_10_labels, test_size=0.2)

In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(spectogram_10_train_data,spectogram_10_train_labels, epochs=50, validation_split=0.2)

In [ ]:
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

spectogram_10_test_loss, spectogram_10_test_acc = model.evaluate(spectogram_10_test_data,  spectogram_10_test_labels)

In [ ]:
print(spectogram_10_test_acc, spectogram_10_test_loss)

In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/spectogram_10_history.csv')

In [ ]:
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/spectogram_10_model.keras')

In [ ]:
del spectogram_10_train_data, spectogram_10_test_data, spectogram_10_train_labels, spectogram_10_test_labels
del history

In [ ]:
!cp '/content/drive/MyDrive/skripsi/eeg/spectogram_20_second_batch' '/content'


In [ ]:
with open('/content/spectogram_20_second_batch', 'rb') as f:
  spectogram_20 = pickle.load(f)

In [ ]:
spectogram_20_data = np.array(spectogram_20[b'data'])
spectogram_20_labels = np.array(spectogram_20[b'labels'])

In [ ]:
spectogram_20_train_data, spectogram_20_test_data, spectogram_20_train_labels, spectogram_20_test_labels = train_test_split(spectogram_20_data, spectogram_20_labels, test_size=0.2)

In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(spectogram_20_train_data,spectogram_20_train_labels, epochs=50, validation_split=0.2)

In [ ]:
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

spectogram_20_test_loss, spectogram_20_test_acc = model.evaluate(spectogram_20_test_data,  spectogram_20_test_labels)

In [ ]:
print(spectogram_20_test_acc, spectogram_20_test_loss)

In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/spectogram_20_history.csv')

In [ ]:
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/spectogram_20_model.keras')

In [ ]:
del spectogram_20_train_data, spectogram_20_test_data, spectogram_20_train_labels, spectogram_20_test_labels
del history

In [ ]:
!cp '/content/drive/MyDrive/skripsi/eeg/spectogram_30_second_batch' '/content'

In [ ]:
with open('/content/spectogram_30_second_batch', 'rb') as f:
  spectogram_30 = pickle.load(f)

In [ ]:
spectogram_30_data = np.array(spectogram_30[b'data'])
spectogram_30_labels = np.array(spectogram_30[b'labels'])

In [ ]:
spectogram_30_train_data, spectogram_30_test_data, spectogram_30_train_labels, spectogram_30_test_labels = train_test_split(spectogram_30_data, spectogram_30_labels, test_size=0.2)

In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(spectogram_30_train_data,spectogram_30_train_labels, epochs=50, validation_split=0.2)

In [ ]:
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

spectogram_30_test_loss, spectogram_30_test_acc = model.evaluate(spectogram_30_test_data,  spectogram_30_test_labels)

In [ ]:
print(spectogram_30_test_acc, spectogram_30_test_loss)

In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/spectogram_30_history.csv')

In [ ]:
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/spectogram_30_model.keras')

In [ ]:
del spectogram_30_train_data, spectogram_30_test_data, spectogram_30_train_labels, spectogram_30_test_labels
del history

In [ ]:
!cp '/content/drive/MyDrive/skripsi/eeg/heatmap_10_second_batch' '/content'

In [ ]:
with open('/content/heatmap_10_second_batch', 'rb') as f:
  heatmap_10 = pickle.load(f)

In [ ]:
heatmap_10_data = np.array(heatmap_10[b'data'])
heatmap_10_labels = np.array(heatmap_10[b'labels'])

In [ ]:
heatmap_10_train_data, heatmap_10_test_data, heatmap_10_train_labels, heatmap_10_test_labels = train_test_split(heatmap_10_data, heatmap_10_labels, test_size=0.2)

In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(heatmap_10_train_data,heatmap_10_train_labels, epochs=50, validation_split=0.2)

In [ ]:
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

heatmap_10_test_loss, heatmap_10_test_acc = model.evaluate(heatmap_10_test_data,  heatmap_10_test_labels)

In [ ]:
print(heatmap_10_test_acc, heatmap_10_test_loss)

In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/heatmap_10_history.csv')

In [ ]:
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/heatmap_10_model.keras')

In [ ]:
del heatmap_10_train_data, heatmap_10_test_data, heatmap_10_train_labels, heatmap_10_test_labels
del history

In [ ]:
!cp '/content/drive/MyDrive/skripsi/eeg/heatmap_20_second_batch' '/content'

In [ ]:
with open('/content/heatmap_20_second_batch', 'rb') as f:
  heatmap_20 = pickle.load(f)

In [ ]:
heatmap_20_data = np.array(heatmap_20[b'data'])
heatmap_20_labels = np.array(heatmap_20[b'labels'])

In [ ]:
heatmap_20_train_data, heatmap_20_test_data, heatmap_20_train_labels, heatmap_20_test_labels = train_test_split(heatmap_20_data, heatmap_20_labels, test_size=0.2)

In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(heatmap_20_train_data,heatmap_20_train_labels, epochs=50, validation_split=0.2)

In [ ]:
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

heatmap_20_test_loss, heatmap_20_test_acc = model.evaluate(heatmap_20_test_data,  heatmap_20_test_labels)

In [ ]:
print(heatmap_20_test_acc, heatmap_20_test_loss)

In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/heatmap_20_history.csv')

In [ ]:
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/heatmap_20_model.keras')

In [ ]:
del heatmap_20_train_data, heatmap_20_test_data, heatmap_20_train_labels, heatmap_20_test_labels
del history

In [ ]:
!cp '/content/drive/MyDrive/skripsi/eeg/heatmap_30_second_batch' '/content'

In [ ]:
with open('/content/heatmap_30_second_batch', 'rb') as f:
  heatmap_30 = pickle.load(f)

In [ ]:
heatmap_30_data = np.array(heatmap_30[b'data'])
heatmap_30_labels = np.array(heatmap_30[b'labels'])

In [ ]:
heatmap_30_train_data, heatmap_30_test_data, heatmap_30_train_labels, heatmap_30_test_labels = train_test_split(heatmap_30_data, heatmap_30_labels, test_size=0.2)

In [ ]:
keras.backend.clear_session(free_memory=True)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


history = model.fit(heatmap_30_train_data,heatmap_30_train_labels, epochs=50, validation_split=0.2)

In [ ]:
fig, axs = plt.subplots(1,1)
axs.plot(history.history['binary_accuracy'], label='accuracy')
axs.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
axs.set_xlabel('Epoch')
axs.set_ylabel('Accuracy')
axs.legend(loc='lower right')

fig1, axs1 = plt.subplots(1,1)
axs1.plot(history.history['loss'], label='loss')
axs1.plot(history.history['val_loss'], label = 'val_loss')
axs1.set_xlabel('Epoch')
axs1.set_ylabel('Loss')
axs1.legend(loc='lower right')

heatmap_30_test_loss, heatmap_30_test_acc = model.evaluate(heatmap_30_test_data,  heatmap_30_test_labels)

In [ ]:
print(heatmap_30_test_acc, heatmap_30_test_loss)

In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/skripsi/eeg/heatmap_30_history.csv')

In [ ]:
keras.models.save_model(model,filepath='/content/drive/MyDrive/skripsi/eeg/heatmap_30_model.keras')

In [ ]:
del heatmap_30_train_data, heatmap_30_test_data, heatmap_30_train_labels, heatmap_30_test_labels
del history